# Creating Project

In [ ]:
import warnings

warnings.filterwarnings("ignore")

from datetime import datetime
import mitzu.model as M


p = M.Project(
    project_name="mysql_project",
    event_data_tables=[
        M.EventDataTable.create(
            table_name="page_events",
            schema="test",
            event_name_field="event_name",
            event_time_field="event_time",
            user_id_field="user_id",
        ),
        M.EventDataTable.create(
            table_name="search_events",
            schema="test",
            event_name_field="event_name",
            event_time_field="event_time",
            user_id_field="user_id",
        ),
    ],
    discovery_settings=M.DiscoverySettings(
        end_dt=datetime(2021, 4, 1),
        lookback_days=84,
        property_sample_rate=100,
    ),
    connection=M.Connection(
        connection_name="mysql_connection",
        connection_type=M.ConnectionType.MYSQL,
        user_name="test",
        secret_resolver=M.ConstSecretResolver("test"),
        host="localhost",
        port="3307",
    ),
)

# Saving Project to File

In [ ]:
dp = p.discover_project()
res = dp.save_to_project_file("mysql_test_project")

# Loading Project

In [ ]:
dp = M.DiscoveredProject.load_from_project_file("mysql_test_project")
m = dp.create_notebook_class_model()

# Product Analytics From Loaded Project

In [ ]:
(m.page_visit >> m.search).config(
    conv_window="7 days", time_group="total", group_by=m.page_visit.domain
).print_sql()

In [ ]:
m.page_visit.config(chart_type="line")

In [ ]:
(m.search >> m.page_visit).config(
    time_group="total",
    conv_window="3 week",
    group_by=m.page_visit.user_country_code,
    lookback_days=120,
)

In [ ]:
(m.search >> m.page_visit).config(
    time_group="total",
    conv_window="3 week",
    group_by=m.page_visit.user_country_code,
    lookback_days=120,
).print_sql()

In [ ]:
# dp.display_inline_dashboard(port=8083)

# or when your notebook does not support iframes
# dp.display_external_dashboard(port=8083)